In [ ]:
import os
import wave
import threading

import pyaudio
from google.cloud import speech, storage

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
YOUR_WAVE = 'audio.wav'

is_recording = False

ModuleNotFoundError: ignored

In [ ]:
def start_record():
    # Record
    global is_recording#進行錄音
    print("start recording ...")#進行錄音
    p = pyaudio.PyAudio()
    s = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    frames = []
    is_recording = True
    while is_recording:#無窮進入聲音
        data = s.read(CHUNK)
        frames.append(data)
    s.stop_stream()#
    s.close()
    p.terminate()
    print("recording done.")
    
    # Save to wave file
    with wave.open(YOUR_WAVE, 'wb') as wav:
        wav.setnchannels(CHANNELS)
        wav.setsampwidth(p.get_sample_size(FORMAT))
        wav.setframerate(RATE)
        wav.writeframes(b''.join(frames))

def stop_record():
    x = input('any key to stop ...')
    global is_recording#設全域變數
    is_recording = False

def recognize():
    YOUR_SERVICE = 'YOUR_SERVICE'
    YOUR_BUCKET = 'YOUR_BUCKET'
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = YOUR_SERVICE
    storage_client = storage.Client()
    speech_client = speech.SpeechClient()
    
    print('start recognizing ...')
    # upload file to GCS(Google Cloud Storage)
    bucket = storage_client.bucket(YOUR_BUCKET)
    bucket.blob(YOUR_WAVE).upload_from_filename(YOUR_WAVE)
    uri = f'gs://{YOUR_BUCKET}/{YOUR_WAVE}'

    # Transcript the audio
    audio = speech.RecognitionAudio(uri=uri)
    config = speech.RecognitionConfig(
        sample_rate_hertz=RATE,
        audio_channel_count=CHANNELS,  # take care, default is 1
        language_code="zh-TW"
    )
    response = speech_client.recognize(config=config, audio=audio)

    for result in response.results:
        print(f'{result.alternatives[0].transcript}, {result.alternatives[0].confidence:.3f}')

In [ ]:
t = threading.Thread(target=start_record)
t.start()
stop_record()
t.join()
recognize()